In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
tokenizer = AutoTokenizer.from_pretrained("luohy/SAIL-7b")

tokenizer_config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "luohy/SAIL-7b",
    device_map="auto",  # Automatically distribute across available GPUs
    torch_dtype=torch.float16  # Use half precision to reduce memory usage
)

config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [5]:
print(model.hf_device_map)

{'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 0, 'model.layers.10': 0, 'model.layers.11': 0, 'model.layers.12': 0, 'model.layers.13': 0, 'model.layers.14': 0, 'model.layers.15': 0, 'model.layers.16': 1, 'model.layers.17': 1, 'model.layers.18': 1, 'model.layers.19': 1, 'model.layers.20': 1, 'model.layers.21': 1, 'model.layers.22': 1, 'model.layers.23': 1, 'model.layers.24': 1, 'model.layers.25': 1, 'model.layers.26': 1, 'model.layers.27': 1, 'model.layers.28': 1, 'model.layers.29': 1, 'model.layers.30': 1, 'model.layers.31': 1, 'model.norm': 1, 'model.rotary_emb': 1, 'lm_head': 1}


now testing on climate

In [8]:
!pip install --upgrade transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 32.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
  Attempting uninstall: datasets
    Found existing installation: datasets 3.3.1
    Uninstalling datasets-3.3.1:
      Successfully uninstalled datasets-3.3.1


In [9]:
from datasets import load_dataset
# climate dataset: https://huggingface.co/datasets/tdiggelm/climate_fever
ds = load_dataset("tdiggelm/climate_fever")

README.md:   0%|          | 0.00/8.09k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/869k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1535 [00:00<?, ? examples/s]

In [10]:
# only have support and refute ds
ds_filtered = ds.filter(lambda example: example["claim_label"] in [0,1])
print(ds_filtered)

Filter:   0%|          | 0/1535 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['claim_id', 'claim', 'claim_label', 'evidences'],
        num_rows: 907
    })
})


"Using only the provided evidence, determine the claim's relation. "
        "Respond with one of these four answers exactly: SUPPORTS, NOT_ENOUGH_INFO, REFUTES, DISPUTED. "
        "Do not add any extra words or explanations.\n\n
acc:0.36
F1: 0.24

Think step by step. Using only the provided evidence, determine the claim's relation. "
    "Respond with one of these four words exactly (and nothing else): SUPPORTS, NOT_ENOUGH_INFO, REFUTES, DISPUTED.\n\n"
    f"Claim: {claim}\n"
Run 1 - Accuracy: 0.3622, F1 Score: 0.2443

Average Accuracy over 10 runs: 0.36221498371335503
Average F1 Score over 10 runs: 0.24430147829908377

prompt = (
        "Lets think step by step. Determine the claim's relation with the evidence provided"
    "Respond with one of these four words exactly: SUPPORTS, NOT_ENOUGH_INFO, REFUTES, DISPUTED.\n\n ."
    f"Claim: {claim}\n"
    )

Run 1 - Accuracy: 0.4104, F1 Score: 0.2356

In [11]:
!pip install openai==0.28


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [ ]:
import openai
import torch
import re
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import accuracy_score, f1_score

def create_prompt(claim):
    return (
        "Is the claim about climate correct or wrong based on your judgement? "
        "Please only answer 'correct' or 'wrong'. One word, no sentence.\n"
        f"Claim: {claim}\n"
        "Answer:"
    )



def extract_after_answer(response_text):
    match = re.search(r'(?i)answer:\s*([A-Za-z]+)\.?', response_text)
    if match:
        answer = re.sub(r'[^\w\s]', '', match.group(1)).lower()
        return 1 if answer == "wrong" else 0
    return 0


def classify_claim_sail(claim, model, tokenizer):
    prompt = create_prompt(claim)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=5)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("SAIL-7b output:", response)
    return extract_after_answer(response)



openai.api_key = "put_your_key_here"
openai.api_base = "https://cmu.litellm.ai/v1"



predictions_sail = []
ground_truths = []


for example in ds_filtered["test"]:
    claim = example["claim"]

    # create GPT-4o-mini "ground truth"
    prompt = create_prompt(claim)
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=50,
        temperature=0.7
    )
    gpt_response_text = response["choices"][0]["message"]["content"]
    print("Answer: ", gpt_response_text)
    gt = extract_after_answer("Answer: "+ gpt_response_text)
    ground_truths.append(gt)

    # 2) SAIL-7b prediction
    pred = classify_claim_sail(claim, model, tokenizer)
    predictions_sail.append(pred)

    # 3) Print for inspection
    print(f"\nClaim: {claim}")
    print("Ground Truth (GPT-4o-mini):", "WRONG" if gt == 1 else "CORRECT")
    print("SAIL-7b Prediction:", "WRONG" if pred == 1 else "CORRECT")
    print("-" * 50)


acc = accuracy_score(ground_truths, predictions_sail)
f1 = f1_score(ground_truths, predictions_sail, average='weighted')

print("SAIL-7b Accuracy vs GPT-4o-mini Ground Truth:", acc)
print("SAIL-7b F1 Score vs GPT-4o-mini Ground Truth:", f1)


Answer:  Correct
SAIL-7b output: Is the claim about climate correct or wrong based on your judgement? Please only answer 'correct' or 'wrong'. One word, no sentence.
Claim: Global warming is driving polar bears toward extinction
Answer: Wrong.

Claim: Global warming is driving polar bears toward extinction
Ground Truth (GPT-4o-mini): CORRECT
SAIL-7b Prediction: WRONG
--------------------------------------------------
Answer:  Wrong
SAIL-7b output: Is the claim about climate correct or wrong based on your judgement? Please only answer 'correct' or 'wrong'. One word, no sentence.
Claim: The sun has gone into ‘lockdown’ which could cause freezing weather, earthquakes and famine, say scientists
Answer: Wrong.

Claim: The sun has gone into ‘lockdown’ which could cause freezing weather, earthquakes and famine, say scientists
Ground Truth (GPT-4o-mini): WRONG
SAIL-7b Prediction: WRONG
--------------------------------------------------
Answer:  Wrong
SAIL-7b output: Is the claim about climate 